In [32]:
# XML 형태의 welfare_data 데이터프레임으로 변경 

import xml.etree.ElementTree as ET
import pandas as pd

tree = ET.parse("welfare_data.txt")
root = tree.getroot()

all_tags = set()
for serv in root.iter("servList"):
    for child in serv:
        all_tags.add(child.tag)

print(f"전체 컬럼 후보: {sorted(all_tags)}")

rows = []
for serv in root.iter("servList"):
    row = {}
    for tag in all_tags:
        row[tag] = serv.findtext(tag)
    rows.append(row)

df = pd.DataFrame(rows)
print(df.head())

전체 컬럼 후보: ['aplyMtdNm', 'bizChrDeptNm', 'ctpvNm', 'inqNum', 'intrsThemaNmArray', 'lastModYmd', 'lifeNmArray', 'servDgst', 'servDtlLink', 'servId', 'servNm', 'sggNm', 'sprtCycNm', 'srvPvsnNm', 'trgterIndvdlNmArray']
  sggNm          bizChrDeptNm  intrsThemaNmArray  \
0  None   광주광역시 경제창업국 농업동물정책과               서민금융   
1  None     대구광역시 시민안전실 건강증진과               서민금융   
2   시흥시     경기도 시흥시 복지국 노인복지과               None   
3   남동구   인천광역시 남동구 복지국 노인정책과               서민금융   
4   청도군  경상북도 청도군 행정복지국 주민복지과  서민금융, 임신·출산, 신체건강   

                                         servDtlLink       servId lastModYmd  \
0  https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfa...  WLF00005111   20250705   
1  https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfa...  WLF00001717   20250705   
2  https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfa...  WLF00000806   20250705   
3  https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfa...  WLF00002697   20250705   
4  https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfa...  WLF00003456   2025

In [35]:
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

True

In [36]:
import os

# API 키 가져오기 (환경 변수에서)
SERVICE_KEY = os.getenv("SERVICE_KEY")

In [38]:
# 상세정보 API 호출 함수

import requests
import xml.etree.ElementTree as ET

def get_service_details(serv_id):
    url = f"http://apis.data.go.kr/B554287/LocalGovernmentWelfareInformations/LcgvWelfaredetailed"
    params = {
        'serviceKey': SERVICE_KEY,
        'servId': serv_id
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        print(f"요청 실패: {response.status_code}")
        return None

    root = ET.fromstring(response.content)
    detail = root
    if detail is None:
        print("상세 정보 없음")
        return None

    data = {}
    skip_tags = {"inqplCtadrList", "baslawList", "basfrmList"}

    for child in detail:
        if child.tag in skip_tags:
            continue
        data[child.tag] = child.text.strip() if child.text else None


    return data

In [39]:
# 호출한 상세내용 데이터프레임에 업데이트 함수

def update_df_with_detail(df, detail_data):
    serv_id = detail_data.get("servId")
    if serv_id is None:
        return df

    row_idx = df[df["servId"] == serv_id].index
    if row_idx.empty:
        print(f"servId {serv_id} not found in df")
        return df

    idx = row_idx[0]

    for key, value in detail_data.items():
        if key == "servId":
            continue

        if key in df.columns:
            if pd.isna(df.at[idx, key]) and value is not None:
                df.at[idx, key] = value
        else:
            df[key] = pd.Series([None] * len(df), dtype=object)  # 👈 이 부분 수정
            df.at[idx, key] = value

    return df

In [40]:
from tqdm import tqdm
import time

def enrich_dataframe_with_details(df, delay=0.3):
    updated_df = df.copy()

    for serv_id in tqdm(updated_df["servId"], desc="업데이트 진행 중", ncols=80):
        detail_data = get_service_details(serv_id)

        if detail_data:
            updated_df = update_df_with_detail(updated_df, detail_data)

        time.sleep(delay)  # 너무 빠른 요청 방지

    return updated_df


In [41]:
df_enriched = enrich_dataframe_with_details(df)

[1/1332] 처리 중: WLF00005111
[2/1332] 처리 중: WLF00001717
[3/1332] 처리 중: WLF00000806
[4/1332] 처리 중: WLF00002697
[5/1332] 처리 중: WLF00003456
[6/1332] 처리 중: WLF00002615
[7/1332] 처리 중: WLF00003160
[8/1332] 처리 중: WLF00000954
[9/1332] 처리 중: WLF00000643
[10/1332] 처리 중: WLF00001873
[11/1332] 처리 중: WLF00001593
[12/1332] 처리 중: WLF00002335
[13/1332] 처리 중: WLF00001045
[14/1332] 처리 중: WLF00000817
[15/1332] 처리 중: WLF00002645
[16/1332] 처리 중: WLF00003159
[17/1332] 처리 중: WLF00001262
[18/1332] 처리 중: WLF00000506
[19/1332] 처리 중: WLF00000504
[20/1332] 처리 중: WLF00003389
[21/1332] 처리 중: WLF00006088
[22/1332] 처리 중: WLF00005931
[23/1332] 처리 중: WLF00005912
[24/1332] 처리 중: WLF00005242
[25/1332] 처리 중: WLF00005113
[26/1332] 처리 중: WLF00005061
[27/1332] 처리 중: WLF00004682
[28/1332] 처리 중: WLF00003859
[29/1332] 처리 중: WLF00003591
[30/1332] 처리 중: WLF00003482
[31/1332] 처리 중: WLF00000692
[32/1332] 처리 중: WLF00003518
[33/1332] 처리 중: WLF00002778
[34/1332] 처리 중: WLF00004048
[35/1332] 처리 중: WLF00001249
[36/1332] 처리 중: WLF00004434
[

In [ ]:
# 겹치는 컬럼 통합 및 삭제

def make_aply_msg(row):
    if pd.notna(row["aplyMtdCn"]):
        return row["aplyMtdCn"]
    elif pd.notna(row["aplyMtdNm"]):
        return f"{row['aplyMtdNm']} 방식으로 신청할 수 있습니다."
    else:
        return "신청 방법에 대한 정보가 제공되지 않았습니다."

df_enriched["신청안내"] = df_enriched.apply(make_aply_msg, axis=1)

In [49]:
# 불필요한 컬럼 삭제

df_enriched.drop(columns=["resultCode", "resultMessage", "inqplHmpgReldList"], inplace=True)
df_enriched.drop(columns=["aplyMtdNm", "aplyMtdCn"], inplace=True)
df_enriched.drop(columns=["inqNum", "lastModYmd"], inplace=True)

In [155]:
# 결측치 해결

# trgterIndvdlNmArray: 대상제한없음
df_enriched['trgterIndvdlNmArray'] = df_enriched['trgterIndvdlNmArray'].fillna('대상제한없음')

# sggNm: ctpvNm 값으로 대체
df_enriched.loc[df_enriched['sggNm'].isnull(), 'sggNm'] = df_enriched.loc[df_enriched['sggNm'].isnull(), 'ctpvNm']

# intrsThemaNmArray: 기타
df_enriched['intrsThemaNmArray'] = df_enriched['intrsThemaNmArray'].fillna('기타')

In [157]:
for col in df_enriched.columns:
    print(f"\n--- {col} ---")
    print(df_enriched[col].value_counts(dropna=False))



--- sggNm ---
sggNm
동구          23
중구          21
서구          17
서귀포시        15
인천광역시       14
            ..
인천광역시교육청     1
의성군          1
예천군          1
금정구          1
대덕구          1
Name: count, Length: 220, dtype: int64

--- bizChrDeptNm ---
bizChrDeptNm
전라남도 구례군 주민복지과             9
강원특별자치도 삼척시 자치행정국 사회복지과    8
전라남도 신안군 문화예술관광국 주민복지과     8
경상남도 하동군 기획행정국 주민행복과       8
전북특별자치도 고창군 자치행정국 사회복지과    8
                          ..
서울특별시 양천구 주민복지국 복지정책과      1
울산광역시 울주군 복지환경국 노인장애인과     1
대구광역시 동구 복지생활국 가족지원과       1
서울특별시 중랑구 생활복지국 사회복지과      1
광주광역시 경제창업국 농업동물정책과        1
Name: count, Length: 715, dtype: int64

--- intrsThemaNmArray ---
intrsThemaNmArray
서민금융                       604
기타                         474
안전·위기                       34
신체건강                        29
서민금융, 입양·위탁                 23
서민금융, 신체건강                  22
서민금융, 일자리                   20
서민금융, 안전·위기                 16
입양·위탁                       11
입양·위탁, 서민금융                  8
일자리                        

In [158]:
# 지원유형 분류

def classify_support_type(text):
    if pd.isna(text):
        return "기타"
    text = str(text)
    if "현금" in text:
        return "현금"
    elif "융자" in text:
        return "융자"
    elif "현물" in text or "물품" in text:
        return "현물"
    elif "지역화폐" in text:
        return "지역화폐"
    elif "전자바우처" in text or "실물바우처" in text:
        return "바우처"
    elif "서비스" in text or "프로그램" in text:
        return "서비스"
    elif "자원봉사" in text:
        return "자원봉사"
    elif "시설입소" in text:
        return "시설입소"
    else:
        return "기타"

# 적용
df_enriched["지원유형"] = df_enriched["srvPvsnNm"].apply(classify_support_type)


In [159]:
# 날짜 형식 변환
df_enriched["enfcBgngYmd"] = pd.to_datetime(df_enriched["enfcBgngYmd"].astype(str), format="%Y%m%d", errors="coerce")
df_enriched["enfcEndYmd"] = pd.to_datetime(df_enriched["enfcEndYmd"].astype(str), format="%Y%m%d", errors="coerce")

# 무기한 여부 판단
df_enriched["무기한여부"] = df_enriched["enfcEndYmd"].dt.year == 9999

# 시행기간 계산
df_enriched["시행기간_년"] = (
    (df_enriched["enfcEndYmd"] - df_enriched["enfcBgngYmd"]).dt.days / 365
).round(1)

# 무기한이면 '무기한', 아니면 시작~종료 표시
df_enriched["시행기간_표기"] = df_enriched.apply(
    lambda row: "무기한" if row["무기한여부"]
    else f"{row['enfcBgngYmd'].date()} ~ {row['enfcEndYmd'].date()}",
    axis=1
)

# 종료 여부 플래그도 함께 (선택)
today = pd.Timestamp.today()
df_enriched["종료됨"] = df_enriched["enfcEndYmd"] < today

In [160]:
# 무기한 여부 재정의: 9999년이거나 NaT인 경우
df_enriched["무기한여부"] = (
    (df_enriched["enfcEndYmd"].dt.year == 9999) | (df_enriched["enfcEndYmd"].isna())
)

# 시행기간 계산 (무기한이면 NaN 유지)
df_enriched["시행기간_년"] = (
    (df_enriched["enfcEndYmd"] - df_enriched["enfcBgngYmd"]).dt.days / 365
).round(1)

# 시행기간 표기
df_enriched["시행기간_표기"] = df_enriched.apply(
    lambda row: "무기한" if row["무기한여부"]
    else f"{row['enfcBgngYmd'].date()} ~ {row['enfcEndYmd'].date()}",
    axis=1
)

# 종료 여부도 다시 정의
today = pd.Timestamp.today()
df_enriched["종료됨"] = (~df_enriched["무기한여부"]) & (df_enriched["enfcEndYmd"] < today)

In [162]:
valid_targets = {
    "저소득", "장애인", "한부모·조손", "보훈대상자", "다문화·탈북민", "다자녀", "대상제한없음"
}

# 리스트화
df_enriched["지원대상_list"] = df_enriched["trgterIndvdlNmArray"].apply(
    lambda lst: sorted([x for x in lst if x in valid_targets]) if isinstance(lst, list) else []
)

# 보기용 문자열 컬럼도 생성
df_enriched["지원대상_표기"] = df_enriched["지원대상_list"].apply(
    lambda x: ", ".join(x) if x else "없음"
)

In [163]:
# 주요 주제군 (사전에 정의)
valid_themas = {
    "서민금융", "생활지원", "일자리", "교육", "보육",
    "신체건강", "임신·출산", "안전·위기", "에너지",
    "입양·위탁", "보호·돌봄", "주거", "문화·여가", "기타"
}

# 리스트화된 컬럼을 기준으로 필터링 및 정제
df_enriched["관심주제_list"] = df_enriched["intrsThemaNmArray"].apply(
    lambda lst: sorted([x for x in lst if x in valid_themas]) if isinstance(lst, list) else []
)

# 보기용 문자열 컬럼 생성
df_enriched["관심주제_표기"] = df_enriched["관심주제_list"].apply(
    lambda x: ", ".join(x) if x else "없음"
)


In [166]:
columns_to_keep = [
    "servId", "servNm", "servDgst", "지원유형", "alwServCn", "sprtTrgtCn", 
    "slctCritCn", "신청안내", "sprtCycNm", "ctpvNm", "sggNm", 
    "lifeNmArray", "trgterIndvdlNmArray", "시행기간_표기", "servDtlLink"
]

df_final = df_enriched[columns_to_keep].copy()

In [168]:
df_final.isnull().sum()

servId                 0
servNm                 0
servDgst               1
지원유형                   0
alwServCn              0
sprtTrgtCn             0
slctCritCn             0
신청안내                   1
sprtCycNm              0
ctpvNm                 0
sggNm                  0
lifeNmArray            0
trgterIndvdlNmArray    0
시행기간_표기                0
servDtlLink            0
dtype: int64

In [169]:
# 결측치가 있는 행 전체 보기
df_final[df_final.isnull().any(axis=1)]

,servId,servNm,servDgst,지원유형,alwServCn,sprtTrgtCn,slctCritCn,신청안내,sprtCycNm,ctpvNm,sggNm,lifeNmArray,trgterIndvdlNmArray,시행기간_표기,servDtlLink
692,WLF00001456,"(임신출산)임신부, 장애인, 노인 전용 엘리베이터 운행",서구청을 찾는 사회적 약자에 대한 배려로 사람중심 도시구현,기타,구청사 내 운휴중인 엘리베이터를 사회적 약자 전용으로 지정 운영\n\n(사업내용)\...,"구청사를 방문하는 임산부, 노인, 장애인","구청사를 방문하는 임산부, 노인, 장애인",NaN,수시,대전광역시,서구,"노년, 영유아",장애인,무기한,https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfa...
1326,WLF00004803,저소득 한부모 가정지원,NaN,현금,"설명절 위문금, 월동대책비 등",한부모가족의 모 또는 부와 만 18세(취학 시 만 22세) 이상의 자녀,기준중위소득 52%이하(청소년한부모 기준중위소득 60%이하),신청 방법에 대한 정보가 제공되지 않았습니다.,년,서울특별시,강동구,"영유아, 아동, 중장년, 노년, 청소년, 청년",한부모·조손,무기한,https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfa...


In [170]:
# 결측치 해결

df_final.loc[692, "신청안내"] = "별도 신청 없이 이용 가능합니다."

desc = "저소득 한부모 가정을 위한 지원사업으로, 명절 위문금 및 월동대책비 등을 지원합니다."
df_final.loc[1326, "servDgst"] = desc

In [174]:
df_final.isnull().sum()

servId                 0
servNm                 0
servDgst               0
지원유형                   0
alwServCn              0
sprtTrgtCn             0
slctCritCn             0
신청안내                   0
sprtCycNm              0
ctpvNm                 0
sggNm                  0
lifeNmArray            0
trgterIndvdlNmArray    0
시행기간_표기                0
servDtlLink            0
dtype: int64

In [175]:
df_final.to_csv('welfare_dataframe.csv', index=False, encoding="utf-8-sig")